In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = "13031026" # 店舗ID
SPREADSHEET_KEY = '1pbHma1UzS_Fv0FmMJbcSZgfPx3H3mVKeq-ffF05I8Oc' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 2001 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+789@gmail.com "
user_pw = "paruparu23"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
s20row = worksheet.cell(11,  28).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

['08/19', '025847', '1', '34359.0', '-12939.0', '74', '19']
['08/19', '025847', '2', '11795.0', '-4513.0', '22', '7']
['08/19', '025847', '3', '34103.0', '-10832.0', '85', '16']
['08/19', '025847', '4', '19487.0', '-1203.0', '63', '8']
['08/19', '025847', '5', '11026.0', '1506.0', '49', '10']
['08/19', '025847', '6', '30256.0', '-10832.0', '71', '9']
['08/19', '025847', '7', '15385.0', '-2707.0', '45', '8']
['08/19', '025847', '8', '10513.0', '603.0', '43', '3']
['08/19', '025338', '9', '37436.0', '-17453.0', '9', '4']
['08/19', '025734', '10', '14103.0', '13543.0', '18', '1']
['08/19', '025734', '11', '10513.0', '-7522.0', '1', '1']
['08/19', '025734', '12', '14359.0', '18357.0', '25', '4']
['08/19', '025750', '13', '38718.0', '13543.0', '99', '5']
['08/19', '025750', '14', '20513.0', '7223.0', '48', '4']
['08/19', '025750', '15', '35385.0', '-12638.0', '47', '5']
['08/19', '025750', '16', '25385.0', '-19559.0', '12', '4']
['08/19', '025395', '17', '21795.0', '-2707.0', '23', '2']
['0

['08/19', '025859', '141', '12821.0', '7524.0', '17', '4']
['08/19', '025670', '142', '25897.0', '7524.0', '49', '4']
['08/19', '025670', '143', '32821.0', '13242.0', '66', '9']
['08/19', '025670', '144', '25641.0', '-17754.0', '5', '2']
['08/19', '025670', '145', '33846.0', '9931.0', '63', '9']
['08/19', '025670', '146', '33333.0', '12941.0', '86', '8']
['08/19', '025670', '147', '31282.0', '15047.0', '62', '5']
['08/19', '025670', '148', '26154.0', '7524.0', '23', '4']
['08/19', '025670', '149', '29487.0', '-3008.0', '48', '6']
['08/19', '025670', '150', '33077.0', '15649.0', '79', '4']
['08/19', '025793', '151', '20513.0', '-1805.0', '35', '8']
['08/19', '025712', '152', '23333.0', '-12337.0', '22', '13']
['08/19', '025664', '153', '38718.0', '13242.0', '99', '14']
['08/19', '025821', '154', '21795.0', '-5717.0', '42', '6']
['08/19', '025779', '155', '30513.0', '-16249.0', '11', '4']
['08/19', '025779', '156', '34103.0', '9330.0', '35', '8']
['08/19', '025856', '1001', '33077.0', '1

['08/19', '025824', '1120', '16154.0', '-9930.0', '4', '2']
['08/19', '025824', '1121', '30513.0', '37316.0', '87', '6']
['08/19', '025824', '1122', '25385.0', '-18356.0', '1', '1']
['08/19', '025606', '1123', '39231.0', '-11735.0', '12', '2']
['08/19', '025606', '1124', '32051.0', '27686.0', '39', '8']
['08/19', '025606', '1125', '31026.0', '15348.0', '28', '5']
['08/19', '025606', '1126', '33590.0', '6320.0', '25', '8']
['08/19', '025606', '1127', '26923.0', '29191.0', '36', '5']
['08/19', '025895', '1128', '49487.0', '-1203.0', '27', '9']
['08/19', '025895', '1129', '47949.0', '34306.0', '48', '6']
['08/19', '025895', '1130', '41282.0', '-902.0', '28', '10']
['08/19', '025895', '1131', '40256.0', '3311.0', '27', '8']
['08/19', '025895', '1132', '49744.0', '-12337.0', '18', '5']
['08/19', '025895', '1133', '50256.0', '29492.0', '50', '7']
['08/19', '025895', '1134', '46154.0', '-2106.0', '30', '10']
['08/19', '025895', '1135', '49487.0', '-27985.0', '13', '5']
['08/19', '025665', '11

['08/19', '120077', '2090', '8368', '2315.0', '0', '39', '80']
['08/19', '120077', '2091', '6448', '3194.0', '0', '39', '64']
['08/19', '120077', '2092', '7111', '-2083.0', '0', '15', '49']
['08/19', '120077', '2093', '8195', '6157.0', '0', '46', '92']
['08/19', '120077', '2094', '7140', '-1157.0', '0', '21', '58']
['08/19', '120077', '2095', '7539', '-2500.0', '0', '17', '46']
['08/19', '120053', '2096', '5754', '-1111.0', '0', '14', '63']
['08/19', '120053', '2097', '8546', '7731.0', '0', '15', '177']
['08/19', '120053', '2098', '5843', '-1065.0', '0', '16', '70']
['08/19', '120053', '2099', '6530', '-139.0', '0', '15', '90']
['08/19', '120053', '2100', '4312', '-694.0', '0', '9', '49']
['08/19', '120053', '2101', '7898', '1019.0', '0', '24', '113']
['08/19', '120053', '2102', '7513', '880.0', '0', '23', '117']
['08/19', '120053', '2103', '7451', '5278.0', '0', '21', '155']
['08/19', '120053', '2104', '7998', '2917.0', '0', '18', '137']
['08/19', '120053', '2105', '6382', '139.0', '0

['08/19', '119996', '3049', '5825', '-694.0', '19', '19', '--']
['08/19', '119996', '3050', '4724', '2222.0', '24', '18', '--']
['08/19', '119996', '3051', '5536', '139.0', '22', '9', '--']
['08/19', '119996', '3052', '2643', '-231.0', '9', '6', '--']
['08/19', '119996', '3053', '6835', '2593.0', '37', '19', '--']
['08/19', '119996', '3054', '7718', '1204.0', '34', '21', '--']
['08/19', '120010', '3055', '4409', '-1296.0', '11', '13', '--']
['08/19', '120010', '3056', '4632', '-139.0', '16', '15', '--']
['08/19', '120010', '3057', '6023', '741.0', '25', '16', '--']
['08/19', '120128', '3058', '7462', '-2407.0', '0', '1', '17']
['08/19', '120128', '3059', '5951', '9815.0', '0', '1', '30']
['08/19', '120128', '3060', '6820', '14769.0', '0', '2', '34']
['08/19', '120128', '3061', '8087', '6204.0', '0', '0', '29']
['08/19', '120128', '3062', '7848', '-2454.0', '0', '1', '21']
['08/19', '120128', '3063', '7572', '556.0', '0', '1', '25']
['08/19', '120128', '3064', '8253', '972.0', '0', '1',

['08/19', '119862', '4022', '4172', '417.0', '15', '0', '16']
['08/19', '120019', '4023', '4110', '-1389.0', '0', '13', '11']
['08/19', '119863', '4024', '3671', '509.0', '30', '0', '24']
['08/19', '119848', '4025', '937', '-231.0', '0', '6', '6']
['08/19', '119821', '4026', '4217', '833.0', '27', '23', '--']
['08/19', '120104', '4027', '3251', '-185.0', '0', '7', '12']
['08/19', '120128', '4028', '6862', '-3056.0', '0', '0', '17']
['08/19', '120122', '4029', '5757', '2361.0', '81', '17', '--']
['08/19', '120153', '4030', '7445', '5694.0', '51', '0', '48']
['08/19', '120148', '4031', '8066', '-1343.0', '0', '15', '45']
['08/19', '120144', '4032', '4543', '-1898.0', '13', '0', '18']
['08/19', '120060', '4033', '5180', '556.0', '61', '21', '--']
['08/19', '119968', '4034', '3459', '-648.0', '16', '13', '--']
['08/19', '119988', '4035', '2365', '-926.0', '0', '6', '2']
['08/19', '120123', '4036', '3843', '185.0', '0', '5', '38']
['08/19', '119993', '4037', '1747', '185.0', '0', '4', '11']

['08/19', '120120', '4153', '6006', '-3148.0', '0', '0', '30']
['08/19', '120120', '4154', '7869', '-741.0', '0', '0', '58']
['08/19', '120104', '4155', '1913', '-1944.0', '0', '0', '4']
['08/19', '120104', '4156', '4362', '-1343.0', '0', '6', '17']
['08/19', '120142', '4157', '9393', '2546.0', '0', '22', '85']
['08/19', '120142', '4158', '5468', '-2269.0', '0', '13', '37']
['08/19', '120142', '4159', '7696', '-2130.0', '0', '15', '57']
['08/19', '120142', '4160', '8265', '-93.0', '0', '22', '81']
['08/19', '120142', '4161', '9510', '3704.0', '0', '27', '89']
['08/19', '120142', '4162', '7936', '-4954.0', '0', '22', '48']
['08/19', '120142', '4163', '7627', '-2407.0', '0', '19', '57']
['08/19', '120142', '4164', '7661', '-4537.0', '0', '18', '44']
['08/19', '120142', '4165', '7151', '1250.0', '0', '15', '61']


{'spreadsheetId': '1pbHma1UzS_Fv0FmMJbcSZgfPx3H3mVKeq-ffF05I8Oc',
 'updates': {'spreadsheetId': '1pbHma1UzS_Fv0FmMJbcSZgfPx3H3mVKeq-ffF05I8Oc',
  'updatedRange': 'S20RAW!F72150:M72640',
  'updatedRows': 491,
  'updatedColumns': 8,
  'updatedCells': 3928}}